In [94]:
import pandas as pd
import numpy as np

import random
import time

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import time 
from scipy import stats


from sklearn.metrics import r2_score

from sklearn.metrics import mean_absolute_percentage_error as mape

#### read the effective species presence-absence

In [95]:
#opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 500,
#                         'RelStopTol': 1e-8, 'AutoRho': {'RsdlTarget': 1.0}})

In [96]:
pa_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-pa.pkl")

In [97]:
labdl = np.round(np.logspace(-5,0.8,5),5)
lam = list(labdl)
lam.append([0.5,0.6,1.0])
lamf = list(pd.DataFrame(lam)[0].explode().values)
lamf.sort()
lamf = lamf[:7]
lamf

[1e-05, 0.00028, 0.00794, 0.22387, 0.5, 0.6, 1.0]

In [98]:
laml = lamf

In [99]:
#pa_redv1

In [100]:
B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")
A = pd.read_pickle("~/bge-analysis-simv3/A.pkl")

tableg=A.T
species_onesall = [] 
for species in range(16):
    species_onesall.append(tableg[tableg[species]==1.0][species].index.to_list())
species_zerosall = []
for species in range(16):
    species_zerosall.append(tableg[tableg[species]==0.0][species].index.to_list())

##### compile the well numbers for each species for its presence 

In [101]:
species_ones = []
for i in range(16):
    species_ones.append(pa_redv1[pa_redv1[i]==1.0]["well number"].values)

In [102]:
ones_len = [] 
for ii in range(len(species_ones)):
    ones_len.append(len(species_ones[ii])) 

In [103]:
steadystate = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-sst-REL.pkl")
sst = steadystate.T

In [104]:
bias_var_listm = [3,3,3,3,3,3,3,3,5,3,5,3,3,7,5,7]

In [105]:
import os
os.getcwd()

'/raid/home/shreyaa5/compressed_sensingv1/realdatasets/OV-v2/2pow15-matrices'

In [107]:
listll = [7,15] 

df = [] 
for kk in listll:
    df.append(pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-v2/2pow15-matrices/16l1-"+str(kk)+"-bge.pkl"))

In [110]:
df2 = pd.concat([df[0],df[1]])
#df2 = pd.concat([df[0],df[1],df[2],df[3],df[4],df[5],df[6]])

dflin = [] 
for kk in listll:
    dflin.append(pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-v2/2pow15-matrices/16l1-"+str(kk)+"-lin.pkl"))

dflin2 = pd.concat([dflin[0],dflin[1]])
#dflin2  = pd.concat([dflin[0],dflin[1],dflin[2],dflin[3],dflin[4],dflin[5],dflin[6]])

In [85]:
###columns = iteras,lmda,species,yrecon[rileft],rileft,rinew,cv_splits,len(rileft),len(rinew)
crossval = [] 
for iteras in range(5):
    rundf = df2[df2[0]==iteras]
    for speciesc in range(16):
        species = speciesc
        sp1 = rundf[rundf[2]==speciesc]
        cv_splits = bias_var_listm[species]
        m = cv_splits 
        subsetall = A[species_onesall[species]].T
        subsetall["indexinwhole"] = subsetall.index
        subsetall.index = np.arange(0,2**15,1)
        givendata = subsetall[subsetall["indexinwhole"].isin(species_ones[species])]
        sstgiven = steadystate.T[subsetall[subsetall["indexinwhole"].isin(species_ones[species])]["indexinwhole"].values].T
        sstgiven.index = givendata.index
        for rhola in laml:
            rho1 = sp1[sp1[1]==rhola]
            r2 = [] 
            mape2 = [] 
            yallbest  = []
            yallact  = [] 
            for iterii in range(m):
                idx = rho1[4].iloc[iterii]
                yact = sstgiven.T[idx].T[species].values
                ybest = rho1[3].iloc[iterii]
                ybestdf1 = pd.DataFrame(ybest)
                ybestdf1[ybestdf1[0]<1e-10]=0
                slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                rval = r2_score(yact,ybestdf1[0].values)
                gop = 1/(2-rval)
                #r2.append([gop,r2_score(yact,ybestdf1[0].values)])
                #mape2.append(mape(yact,ybestdf1[0].values))
                yallbest.append(ybestdf1[0].values)
                yallact.append(yact)
                #slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                #rval = r2_score(yact,ybestdf1[0].values)
                #gop = 1/(2-rval)
                mape1 = mape(yact,ybestdf1[0].values)
                r2.append([gop,rval,mape1,slope,intercept,r_value**2])
            r2df = pd.DataFrame(r2)
            if m == 2:
                ybestagg = [*yallbest[0],*yallbest[1]]
                yactagg = [*yallact[0],*yallact[1]]
            if m == 3:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2]]
            if m == 5:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
            if m == 6:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5]]
            if m == 7:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
            if m == 9:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8]]
            slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
            rval2 = r2_score(yactagg,ybestagg)
            gop2 = 1/(2-rval2)
            mean1 = r2df.mean().values[0]
            mean2 = r2df.mean().values[1]
            mean3 = r2df.mean().values[2]
            mean4 = r2df.mean().values[3]
            mean5 = r2df.mean().values[4]
            mean6 = r2df.mean().values[5]
            mape2 = mape(yactagg,ybestagg)
            crossval.append([iteras,m,speciesc,rhola,gop2,rval2,mape2,slope,intercept,r_value**2,"break",mean1,mean2,mean3,mean4,mean5,mean6,yactagg,ybestagg])
    

In [87]:
crossdf = pd.DataFrame(crossval,columns=["run","k-fold","sp","lambda","nnseagg","r2agg","mapeagg","slagg","intagg","rlinagg","means on right","nnseavg","r2avg","mapeavg","slavg","intavg","rlinavg","testact","testpred"])

In [81]:
pdf16 = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-v2/WH-REL-analysis.pkl")
spp = pdf16[pdf16["sp"]==3]
spp[spp["k-fold"]==3]

,k-fold,sp,lambda,nnseagg,r2agg,mapeagg,slagg,intagg,rlinagg,means on right,nnseavg,r2avg,mapeavg,slavg,intavg,rlinavg,testact,testpred
133,3,3,0.00001,0.802197,0.753423,0.167325,0.837782,0.115167,0.761935,break,0.797361,0.734409,0.167325,8.403720e-01,0.110825,8.177988e-01,"[0.99683500756846, 0.7273075817118758, 0.45096...","[0.8281296935681465, 0.8111683522840455, 0.329..."
134,3,3,0.00028,0.810503,0.766198,0.165663,0.865894,0.103456,0.779740,break,0.805127,0.740009,0.165663,8.646899e-01,0.101816,8.212910e-01,"[0.99683500756846, 0.7273075817118758, 0.45096...","[0.8405476912407784, 0.8313826263576128, 0.313..."
135,3,3,0.00794,0.821756,0.783093,0.155314,0.878326,0.089745,0.793628,break,0.816727,0.758564,0.155314,8.715800e-01,0.090626,8.333539e-01,"[0.99683500756846, 0.7273075817118758, 0.45096...","[0.8244311826404873, 0.818518286691907, 0.3302..."
136,3,3,0.22387,0.701410,0.574299,0.187307,0.385588,0.385704,0.808465,break,0.705461,0.573339,0.187307,3.939874e-01,0.381134,9.081067e-01,"[0.99683500756846, 0.7273075817118758, 0.45096...","[0.8153024822537606, 0.7280571813609852, 0.491..."
137,3,3,0.50000,0.532470,0.121960,0.279325,0.099056,0.560832,0.242183,break,0.538695,0.126804,0.279325,1.101689e-01,0.554788,5.225247e-01,"[0.99683500756846, 0.7273075817118758, 0.45096...","[0.6931241074784815, 0.6931241074784815, 0.532..."
138,3,3,0.60000,0.511910,0.046531,0.291373,0.058742,0.583656,0.123930,break,0.514103,0.049353,0.291373,7.030632e-02,0.576410,3.832806e-01,"[0.99683500756846, 0.7273075817118758, 0.45096...","[0.6633943987293898, 0.6633943987293898, 0.540..."
139,3,3,1.00000,0.461097,-0.168742,0.314906,-0.009184,0.591028,0.018367,break,0.463098,-0.166141,0.314906,-2.074820e-32,0.584892,2.542905e-32,"[0.99683500756846, 0.7273075817118758, 0.45096...","[0.6017595247611749, 0.6017595247611749, 0.601..."


In [89]:
###columns = iteras,lmda,species,yrecon[rileft],rileft,rinew,cv_splits,len(rileft),len(rinew)
crossvalin = [] 
dflin = dflin2
for iteras in range(5):
    rundf = dflin[dflin[0]==iteras]
    for speciesc in range(16):
        species = speciesc
        sp1 = rundf[rundf[2]==speciesc]
        cv_splits = bias_var_listm[species]
        m = cv_splits 
        subsetall = A[species_onesall[species]].T
        subsetall["indexinwhole"] = subsetall.index
        subsetall.index = np.arange(0,2**15,1)
        givendata = subsetall[subsetall["indexinwhole"].isin(species_ones[species])]
        sstgiven = steadystate.T[subsetall[subsetall["indexinwhole"].isin(species_ones[species])]["indexinwhole"].values].T
        sstgiven.index = givendata.index
        for rhola in laml:
            rho1 = sp1[sp1[1]==rhola]
            r2 = [] 
            mape2 = [] 
            yallbest  = []
            yallact  = [] 
            for iterii in range(m):
                idx = rho1[4].iloc[iterii]
                yact = sstgiven.T[idx].T[species].values
                ybest = rho1[3].iloc[iterii]
                ybestdf1 = pd.DataFrame(ybest)
                ybestdf1[ybestdf1[0]<1e-10]=0
                slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                rval = r2_score(yact,ybestdf1[0].values)
                gop = 1/(2-rval)
                #r2.append([gop,r2_score(yact,ybestdf1[0].values)])
                #mape2.append(mape(yact,ybestdf1[0].values))
                yallbest.append(ybestdf1[0].values)
                yallact.append(yact)
                #slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                #rval = r2_score(yact,ybestdf1[0].values)
                #gop = 1/(2-rval)
                mape1 = mape(yact,ybestdf1[0].values)
                r2.append([gop,rval,mape1,slope,intercept,r_value**2])
            r2df = pd.DataFrame(r2)
            if m == 2:
                ybestagg = [*yallbest[0],*yallbest[1]]
                yactagg = [*yallact[0],*yallact[1]]
            if m == 3:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2]]
            if m == 5:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
            if m == 6:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5]]
            if m == 7:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
            if m == 9:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8]]
            slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
            rval2 = r2_score(yactagg,ybestagg)
            gop2 = 1/(2-rval2)
            mean1 = r2df.mean().values[0]
            mean2 = r2df.mean().values[1]
            mean3 = r2df.mean().values[2]
            mean4 = r2df.mean().values[3]
            mean5 = r2df.mean().values[4]
            mean6 = r2df.mean().values[5]
            mape2 = mape(yactagg,ybestagg)
            crossvalin.append([iteras,m,speciesc,rhola,gop2,rval2,mape2,slope,intercept,r_value**2,"break",mean1,mean2,mean3,mean4,mean5,mean6,yactagg,ybestagg])
    

In [91]:
crosslindf = pd.DataFrame(crossvalin,columns=["run","k-fold","sp","lambda","nnseagg","r2agg","mapeagg","slagg","intagg","rlinagg","means on right","nnseavg","r2avg","mapeavg","slavg","intavg","rlinavg","testact","testpred"])

In [93]:
pd.to_pickle(crossdf,"WHpow15-REL-analysis.pkl")
pd.to_pickle(crosslindf,"sigmapow15-REL-analysis.pkl")

In [174]:
meanmaster = [] 
for species in range(16):
    spmean = [] 
    spps = crossdf[crossdf["sp"]==species]
    for runs in range(5):
        rundf = spps[spps["run"]==runs]
        spmean.append(rundf["nnseagg"].max())
        #print(rundf.T[rundf["nnseagg"].idxmax()]["lambda"])
    meanmaster.append([species,np.asarray(spmean).mean()])

In [171]:
rundf["nnseagg"].idxmax()

457

In [175]:
pd.DataFrame(meanmaster)

,0,1
0,0,0.617813
1,1,0.835846
2,2,0.790251
3,3,0.704611
4,4,0.615495
5,5,0.505057
6,6,0.759360
7,7,0.605959
8,8,0.505512
9,9,0.449431


In [166]:
splist = pd.DataFrame(['BV', 'BF', 'BT', 'BC', 'BY', 'DP', 'BL', 'BA', 'FP', 'CH', 'AC','BH', 'CG', 'ER', 'RI', 'CC'])

In [176]:
splist["per"] = pd.DataFrame(meanmaster)[1]

In [179]:
splist.sort_values(by="per",ascending=False)

,0,per
15,CC,0.854532
1,BF,0.835846
2,BT,0.790251
6,BL,0.759360
10,AC,0.750238
12,CG,0.720226
3,BC,0.704611
13,ER,0.663130
0,BV,0.617813
4,BY,0.615495


In [183]:
splist.T[[1,2,13,15,10]]

,1,2,13,15,10
0,BF,BT,ER,CC,AC
per,0.835846,0.790251,0.66313,0.854532,0.750238
